
# BIOINFORMATIKA - deceptív fitneszfüggvény gyakorlati alkalmazása
**Feladat:** deceptív fitneszfüggvény bevezetés és gyakorlati alkalmazás

---

Deceptív függvény:
* olyan fitness függvény, amelyben a lokálisan növekvő sémák (schema-average fitness) a GA-t a globális optimumtól távolodó irányba vezetik, így a GA jellemzően rossz lokális optimumhoz konvergál.


* globális maximum: 0000
* bármely alacsonyabb rendű szkémára teljesül: ha az összes 0-t 1-re cseréljük, akkor a szkéma átlagfitnesze nő.

* Készítünk egy 8 bites függvényt:
* f8​(x1​,…,x8​)=f4​(x1​,…,x4​)+f4​(x5​,…,x8​), ahol:
* az első 4 bit első blokknak, az utolsó 4 bit második blokknak felel meg.

Globális maximum:
* Az első blokk maximuma 0000, a másodiké is 0000,
* ezért az egész 8 bites maximumpont: 00000000.

Miért deceptív? (építőkocka-értelemben)
* Minden 4 bites blokk külön-külön deceptív (ezt az első projekt feladatban kell majd bizonyítani)
* Egy 8 bites sémánál, ha egy blokkban kicseréljük a 0-kat 1-re, akkor a blokk átlag-fitnesze nő → a teljes függvény blokkra eső része is nő.
* Így a GA-t bitenként, blokkonként „félrevezeti”: a lokális mintázat az 1-esek irányába tol, miközben a globális optimum a teljesen 0-s vektor.

---



## LABOR / FELADATOK CÉLJA

*   Genetikus algoritmus alapelveinek megértése
*   Szkémák (schema) és átlagfitnesz fogalmának gyakorlása
*   Deceptív fitness táj viselkedésének megismerése
*   Kódolás és problémareprezentáció szerepének megértése
*   Paraméterérzékenység vizsgálata
*   Lokális vs. globális optimum fogalmának elmélyítése
*   Kísérleti (empirikus) gondolkodás fejlesztése
*   Kódolási és implementációs készségek fejlesztése Pythonban
*   Kritikus gondolkodás az algoritmusválasztásról

## FELADAT: KÓD MEGÉRTÉS, majd gyakorlatok extra feladatokkal
A saját implementációhoz adok egy alap kódót, hogy ha vki elakadna, legyen minimál segítség, de ez legyen csak iránymutatás a saját megoldáshoz.

In [ ]:
from itertools import product

# 4 bites Whitley-féle DF2 (globális max: 1111, lokális: 0000)
DF2 = {
    "0000": 28, "0001": 26, "0010": 24, "0011": 18,
    "0100": 22, "0101":  6, "0110": 14, "0111":  0,
    "1000": 20, "1001": 12, "1010": 10, "1011":  2,
    "1100":  8, "1101":  4, "1110":  6, "1111": 30,
}

def bit_not(bits: str) -> str:
    return "".join("1" if b == "0" else "0" for b in bits)

# 4 bites deceptív függvény, ahol a globális maximum 0000
f4 = {b: DF2[bit_not(b)] for b in DF2}

def f8(bits: str) -> int:
    """8 bites deceptív fitness: két 4 bites blokk összege."""
    assert len(bits) == 8
    return f4[bits[:4]] + f4[bits[4:]]

# Ellenőrzés: keressük meg a globális maximumot
BITSTRINGS_8 = ["".join(b) for b in product("01", repeat=8)]
values = {x: f8(x) for x in BITSTRINGS_8}
best_val = max(values.values())
best_points = [x for x, v in values.items() if v == best_val]

print("Globális maximum érték (8 bit):", best_val)
print("Globális maximum helye(i):", best_points)  # elvárt: ['00000000']


Globális maximum érték (8 bit): 60
Globális maximum helye(i): ['00000000']


## MELEGÍTŐ FELADAT (könnyű)



1.   Futtassátok a fenti kódot, győződjetek meg róla, hogy a globális maximum tényleg 00000000.
2.   Írjatok GA-t, és figyeljék meg, hogy tipikusan „1-esek felé” sodródik a populáció, nem pedig a 00000000 irányába.





In [ ]:
import random

POPULATION_SIZE = 100
GENERATIONS = 50
MUTATION_RATE = 0.01
CHROMOSOME_LENGHT = 8

def generate_individual(length: int) -> str:
    """Véletlenszerű 8 bites egyed generálása."""
    return ''.join(random.choice('01') for _ in range(length))

def initialize_population(size: int, chromosome_length: int) -> list:
    """Kezdő populáció inicializálása."""
    return [generate_individual(chromosome_length) for _ in range(size)]

def calculate_population_fitness(population: list, fitness_func) -> list:
    """A populáció minden egyedének fitneszét kiszámítja."""
    return [(individual, fitness_func(individual)) for individual in population]

def select_parents_roulette_wheel(evaluated_population: list) -> list:
    """Rulettkerék szelekció a szülők kiválasztására."""
    total_fitness = sum(fitness for _, fitness in evaluated_population)
    if total_fitness == 0:
        return random.choices([ind for ind, _ in evaluated_population], k=POPULATION_SIZE)

    selection_probs = [fitness / total_fitness for _, fitness in evaluated_population]
    selected_indices = random.choices(range(len(evaluated_population)), weights=selection_probs, k=POPULATION_SIZE)
    return [evaluated_population[i][0] for i in selected_indices]

def crossover(parent1: str, parent2: str) -> tuple[str, str]:
    """Egypontos keresztezés."""
    crossover_point = random.randint(1, len(parent1) - 1)
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2

def mutate(individual: str, mutation_rate: float) -> str:
    """Bit-flip mutáció."""
    mutated_individual = list(individual)
    for i in range(len(mutated_individual)):
        if random.random() < mutation_rate:
            mutated_individual[i] = '1' if mutated_individual[i] == '0' else '0'
    return "".join(mutated_individual)

population = initialize_population(POPULATION_SIZE, CHROMOSOME_LENGHT)

print(f"\n--- Genetikus Algoritmus Futtatása ({CHROMOSOME_LENGHT} bit) ---")
for generation in range(GENERATIONS):
    evaluated_population = calculate_population_fitness(population, f8)
    best_individual, best_fitness = max(evaluated_population, key=lambda item: item[1])
    avg_fitness = sum(fitness for _, fitness in evaluated_population) / POPULATION_SIZE

    ones_in_best = best_individual.count('1')

    print(f"Generáció {generation + 1}: Legjobb fitnesz: {best_fitness}, Legjobb egyed: {best_individual} (1-esek száma: {ones_in_best}), Átlag fitnesz: {avg_fitness:.2f}")

    new_population = []
    parents = select_parents_roulette_wheel(evaluated_population)

    while len(new_population) < POPULATION_SIZE:
        parent1 = random.choice(parents)
        parent2 = random.choice(parents)

        child1, child2 = crossover(parent1, parent2)

        new_population.append(mutate(child1, MUTATION_RATE))
        if len(new_population) < POPULATION_SIZE:
            new_population.append(mutate(child2, MUTATION_RATE))
    population = new_population

evaluated_population = calculate_population_fitness(population, f8)
final_best_individual, final_best_fitness = max(evaluated_population, key=lambda item: item[1])
final_avg_fitness = sum(fitness for _, fitness in evaluated_population) / POPULATION_SIZE
final_ones_in_best = final_best_individual.count('1')
print(f"\n--- Eredmények ---")
print(f"Végső legjobb egyed: {final_best_individual}, Fitnesz: {final_best_fitness} (1-esek száma: {final_ones_in_best})")
print(f"Végső átlag fitnesz: {final_avg_fitness:.2f}")
if final_best_individual == '00000000':
    print("A GA megtalálta a globális optimumot (00000000).")
else:
    print("A GA nem találta meg a globális optimumot (00000000), valószínűleg a '1'-esek felé sodródott.")



--- Genetikus Algoritmus Futtatása (8 bit) ---
Generáció 1: Legjobb fitnesz: 58, Legjobb egyed: 11110000 (1-esek száma: 4), Átlag fitnesz: 29.16
Generáció 2: Legjobb fitnesz: 58, Legjobb egyed: 11110000 (1-esek száma: 4), Átlag fitnesz: 33.40
Generáció 3: Legjobb fitnesz: 58, Legjobb egyed: 11110000 (1-esek száma: 4), Átlag fitnesz: 38.02
Generáció 4: Legjobb fitnesz: 58, Legjobb egyed: 11110000 (1-esek száma: 4), Átlag fitnesz: 40.72
Generáció 5: Legjobb fitnesz: 58, Legjobb egyed: 00001111 (1-esek száma: 4), Átlag fitnesz: 43.10
Generáció 6: Legjobb fitnesz: 56, Legjobb egyed: 11111111 (1-esek száma: 8), Átlag fitnesz: 42.16
Generáció 7: Legjobb fitnesz: 56, Legjobb egyed: 11111111 (1-esek száma: 8), Átlag fitnesz: 44.22
Generáció 8: Legjobb fitnesz: 56, Legjobb egyed: 11111111 (1-esek száma: 8), Átlag fitnesz: 45.00
Generáció 9: Legjobb fitnesz: 56, Legjobb egyed: 11111111 (1-esek száma: 8), Átlag fitnesz: 46.50
Generáció 10: Legjobb fitnesz: 56, Legjobb egyed: 11111111 (1-esek szá

## 1) FELADAT (könnyű) – Táblázat kitöltése 4 biten

*   Írd fel táblázatba az összes 4 bites kromoszómát, a hozzájuk tartozó 𝑓4 fitneszértéket.
*   Rendezd őket csökkenő sorrendbe a fitness szerint.
*   Jelöld, hol van a globális maximum és hol a „csapda” (lokális optimum).



In [ ]:
from itertools import product

BITSTRINGS_4 = ["".join(b) for b in product("01", repeat=4)]

fitness_data = []
for bits in BITSTRINGS_4:
    fitness_data.append((bits, f4[bits]))

fitness_data.sort(key=lambda item: item[1], reverse=True)

print("4 bites kromoszómák és fitneszértékeik (csökkenő sorrendben)")
print(f"{'Kromoszoma':<10} | {'Fitnesz':<8} | Megjegyzés")
print("-" * 35)


global_max_chromosome = '0000'
trap_local_optimum = '1111'

for chromosome, fitness in fitness_data:
    note = ""
    if chromosome == global_max_chromosome:
        note = "Globális maximum"
    elif chromosome == trap_local_optimum:
        note = "Csapda (lokális optimum)"

    print(f"{chromosome:<10} | {fitness:<8} | {note}")

4 bites kromoszómák és fitneszértékeik (csökkenő sorrendben)
Kromoszoma | Fitnesz  | Megjegyzés
-----------------------------------
0000       | 30       | Globális maximum
1111       | 28       | Csapda (lokális optimum)
1110       | 26       | 
1101       | 24       | 
1011       | 22       | 
0111       | 20       | 
1100       | 18       | 
1001       | 14       | 
0110       | 12       | 
0101       | 10       | 
0011       | 8        | 
0001       | 6        | 
1010       | 6        | 
0010       | 4        | 
0100       | 2        | 
1000       | 0        | 


## 2) FELADAT (könnyű) – Séma-átlag kézzel

*   Válaszd a sémát: 0***, számold ki kézzel a hozzá tartozó átlagfitneszt.
*   Utána nézd meg a 1*** sémát, és hasonlítsd össze.
*   Kövesd végig még 2–3 sémára (pl. 00** → 11**, * 0 * 0 → * 1 * 1 ).


In [ ]:
def calculate_schema_average_fitness(schema: str, fitness_func) -> float:
    """Kiszámolja egy adott séma átlagfitneszét."""
    matching_individuals = []
    for individual in BITSTRINGS_4:
        matches = True
        for i in range(len(schema)):
            if schema[i] != '*' and schema[i] != individual[i]:
                matches = False
                break
        if matches:
            matching_individuals.append(individual)

    if not matching_individuals:
        return 0.0

    total_fitness = sum(fitness_func[ind] for ind in matching_individuals)
    return total_fitness / len(matching_individuals)

print("Séma-átlag fitneszek számítása")

schema_0 = "0***"
schema_1 = "1***"

schema_00 = "00**"
schema_11 = "11**"

schema_0_0 = "*0*0"
schema_1_1 = "*1*1"

avg_fitness_0 = calculate_schema_average_fitness(schema_0, f4)
avg_fitness_1 = calculate_schema_average_fitness(schema_1, f4)

avg_fitness_00 = calculate_schema_average_fitness(schema_00, f4)
avg_fitness_11 = calculate_schema_average_fitness(schema_11, f4)

avg_fitness_star0star0 = calculate_schema_average_fitness(schema_0_0, f4)
avg_fitness_star1star1 = calculate_schema_average_fitness(schema_1_1, f4)

print(f"Séma '{schema_0}': Átlagfitnesz = {avg_fitness_0:.2f}")
print(f"Séma '{schema_1}': Átlagfitnesz = {avg_fitness_1:.2f}")
print(f"Összehasonlítás: A '{schema_1}' átlagfitnesze ({avg_fitness_1:.2f}) magasabb, mint a '{schema_0}' átlagfitnesze ({avg_fitness_0:.2f}), pedig a globális optimum a 0-val kezdődik. Ez félrevezeti a GA-t.")
print("\n")

print(f"Séma '{schema_00}': Átlagfitnesz = {avg_fitness_00:.2f}")
print(f"Séma '{schema_11}': Átlagfitnesz = {avg_fitness_11:.2f}")
print(f"Összehasonlítás: Hasonlóan, a '{schema_11}' átlagfitnesze ({avg_fitness_11:.2f}) magasabb, mint a '{schema_00}' átlagfitnesze ({avg_fitness_00:.2f}).")
print("\n")

print(f"Séma '{schema_0_0}': Átlagfitnesz = {avg_fitness_star0star0:.2f}")
print(f"Séma '{schema_1_1}': Átlagfitnesz = {avg_fitness_star1star1:.2f}")
print(f"Összehasonlítás: Itt is megfigyelhető, hogy a '{schema_1_1}' átlagfitnesze ({avg_fitness_star1star1:.2f}) magasabb, mint a '{schema_0_0}' átlagfitnesze ({avg_fitness_star0star0:.2f}).")

print("\n--- Értelmezés ---")
print("A fenti eredmények alapján jól látható, hogy az összes vizsgált esetben a '1'-eseket nagyobb arányban tartalmazó sémák átlagfitnesze magasabb, mint a '0'-kat nagyobb arányban tartalmazóké. Ez azt jelenti, hogy a genetikus algoritmus, amely a magasabb fitneszű egyedek szelekciójára épül, a populációt a '1'-esek irányába fogja terelni, még akkor is, ha a globális optimum (0000) '0'-kból áll. Ez a deceptív fitneszfüggvény jellemző viselkedése, ami megakadályozza a GA-t a globális optimum megtalálásában a konvencionális megközelítésekkel.")

Séma-átlag fitneszek számítása
Séma '0***': Átlagfitnesz = 11.50
Séma '1***': Átlagfitnesz = 17.25
Összehasonlítás: A '1***' átlagfitnesze (17.25) magasabb, mint a '0***' átlagfitnesze (11.50), pedig a globális optimum a 0-val kezdődik. Ez félrevezeti a GA-t.


Séma '00**': Átlagfitnesz = 12.00
Séma '11**': Átlagfitnesz = 24.00
Összehasonlítás: Hasonlóan, a '11**' átlagfitnesze (24.00) magasabb, mint a '00**' átlagfitnesze (12.00).


Séma '*0*0': Átlagfitnesz = 10.00
Séma '*1*1': Átlagfitnesz = 20.50
Összehasonlítás: Itt is megfigyelhető, hogy a '*1*1' átlagfitnesze (20.50) magasabb, mint a '*0*0' átlagfitnesze (10.00).

--- Értelmezés ---
A fenti eredmények alapján jól látható, hogy az összes vizsgált esetben a '1'-eseket nagyobb arányban tartalmazó sémák átlagfitnesze magasabb, mint a '0'-kat nagyobb arányban tartalmazóké. Ez azt jelenti, hogy a genetikus algoritmus, amely a magasabb fitneszű egyedek szelekciójára épül, a populációt a '1'-esek irányába fogja terelni, még akkor is, ha

## 3) FELADAT (könnyű) – GA egy generáció kézzel

*   Adj meg egy 6 elemű populációt 4 bites kromoszómákkal.
*   Számold ki a fitness értékeket, majd **roulette-wheel szelekcióval** válaszd ki 6 utód „szülőt”.
*   Nincs keresztezés, nincs mutáció – csak nézd meg, hogyan tolódik a populáció eloszlása.

Megjegyzés: Roulette-szelekcióra korábban adtam már példát. Azt lehet használni, mint inspirációt.



In [ ]:
import random

def select_parents_roulette_wheel_custom_k(evaluated_population: list, k: int) -> list:
    """Rulettkerék szelekció a szülők kiválasztására, egyedi 'k' számú kiválasztással."""
    total_fitness = sum(fitness for _, fitness in evaluated_population)
    if total_fitness == 0:
        return random.choices([ind for ind, _ in evaluated_population], k=k)

    selection_probs = [fitness / total_fitness for _, fitness in evaluated_population]
    selected_indices = random.choices(range(len(evaluated_population)), weights=selection_probs, k=k)
    return [evaluated_population[i][0] for i in selected_indices]

population_size_task3 = 6
initial_population_task3 = [
    "0001",
    "1110",
    "0101",
    "1011",
    "1111",
    "0000"
]

print("Kezdeti populácio:")
for individual in initial_population_task3:
    print(f"  {individual} (Fitnesz: {f4[individual]}) ")

evaluated_population_task3 = [(ind, f4[ind]) for ind in initial_population_task3]

selected_parents_task3 = select_parents_roulette_wheel_custom_k(evaluated_population_task3, population_size_task3)

print("\nRoulette-wheel szelekcióval kiválasztott utód 'szülők':")
for parent in selected_parents_task3:
    print(f"  {parent} (Fitnesz: {f4[parent]}) ")

Kezdeti populáció:
  0001 (Fitnesz: 6) 
  1110 (Fitnesz: 26) 
  0101 (Fitnesz: 10) 
  1011 (Fitnesz: 22) 
  1111 (Fitnesz: 28) 
  0000 (Fitnesz: 30) 

Roulette-wheel szelekcióval kiválasztott utód 'szülők':
  0000 (Fitnesz: 30) 
  1110 (Fitnesz: 26) 
  0001 (Fitnesz: 6) 
  1110 (Fitnesz: 26) 
  1110 (Fitnesz: 26) 
  0101 (Fitnesz: 10) 


A kiválasztott 'szülők' listája megmutatja, hogy a magasabb fitneszű egyedek (pl. 0000, 1111, 1110) nagyobb eséllyel kerültek be a következő generációba (azonos kromoszoma néven). Ez a szelekciós nyomás segíti a populációt a fitnesztáj felderítésében, de a deceptív fitneszfüggvény miatt az 1-eseket tartalmazó, lokálisan kedvezőbb sémák felé tolódhat el.

## 4) FELADAT (közepes) – GA implementáció 4 bitre


*   Írj egy egyszerű GA-t 4 bitre (pl. populáció=10, generációk=20).
*   Keresztezés: egypontos, mutáció: bit-flip 1%.
*   Figyeld meg, hányszor találja meg a 0000-t 50 futásból (statisztika).



In [ ]:
import random
from itertools import product

def generate_individual(length: int) -> str:
    """Véletlenszerű 'length' bites egyed generálása."""
    return ''.join(random.choice('01') for _ in range(length))

def initialize_population(size: int, chromosome_length: int) -> list:
    """Kezdő populáció inicializálása."""
    return [generate_individual(chromosome_length) for _ in range(size)]

def calculate_population_fitness(population: list, fitness_func) -> list:
    """A populáció minden egyedének fitneszét kiszámítja."""
    return [(individual, fitness_func[individual]) for individual in population]

def select_parents_roulette_wheel(evaluated_population: list, pop_size: int) -> list:
    """Rulettkerék szelekcio a szülők kiválasztására."""
    total_fitness = sum(fitness for _, fitness in evaluated_population)
    if total_fitness == 0:
        return random.choices([ind for ind, _ in evaluated_population], k=pop_size)

    selection_probs = [fitness / total_fitness for _, fitness in evaluated_population]
    selected_indices = random.choices(range(len(evaluated_population)), weights=selection_probs, k=pop_size)
    return [evaluated_population[i][0] for i in selected_indices]

def crossover(parent1: str, parent2: str) -> tuple[str, str]:
    """Egypontos keresztezés."""
    crossover_point = random.randint(1, len(parent1) - 1)
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2

def mutate(individual: str, mutation_rate: float) -> str:
    """Bit-flip mutáció."""
    mutated_individual = list(individual)
    for i in range(len(mutated_individual)):
        if random.random() < mutation_rate:
            mutated_individual[i] = '1' if mutated_individual[i] == '0' else '0'
    return "".join(mutated_individual)

POPULATION_SIZE_4BIT = 10
GENERATIONS_4BIT = 20
MUTATION_RATE_4BIT = 0.01
CHROMOSOME_LENGTH_4BIT = 4
NUM_SIMULATIONS = 50
GLOBAL_OPTIMUM_4BIT = '0000'

found_optimum_count = 0

for i in range(NUM_SIMULATIONS):
    population = initialize_population(POPULATION_SIZE_4BIT, CHROMOSOME_LENGTH_4BIT)

    for generation in range(GENERATIONS_4BIT):
        evaluated_population = calculate_population_fitness(population, f4)
        best_individual, best_fitness = max(evaluated_population, key=lambda item: item[1])

        if best_individual == GLOBAL_OPTIMUM_4BIT:
            break

        new_population = []
        parents = select_parents_roulette_wheel(evaluated_population, POPULATION_SIZE_4BIT)

        while len(new_population) < POPULATION_SIZE_4BIT:
            parent1 = random.choice(parents)
            parent2 = random.choice(parents)

            child1, child2 = crossover(parent1, parent2)

            new_population.append(mutate(child1, MUTATION_RATE_4BIT))
            if len(new_population) < POPULATION_SIZE_4BIT:
                new_population.append(mutate(child2, MUTATION_RATE_4BIT))
        population = new_population

    evaluated_population = calculate_population_fitness(population, f4)
    final_best_individual, _ = max(evaluated_population, key=lambda item: item[1])

    if final_best_individual == GLOBAL_OPTIMUM_4BIT:
        found_optimum_count += 1

    print(f"  Szimulácio {i+1}/{NUM_SIMULATIONS}: Végső legjobb egyed: {final_best_individual}", end="\r")

print("\n\n Eredmények:")
print(f"A GA {NUM_SIMULATIONS} futásból {found_optimum_count} alkalommal találta meg a globális optimumot ({GLOBAL_OPTIMUM_4BIT}).")


  Szimulácio 50/50: Végső legjobb egyed: 1111

 Eredmények:
A GA 50 futásból 20 alkalommal találta meg a globális optimumot (0000).


## 5) FELADAT (közepes) – Sémaszintű vizsgálat (8 bit)


*   A f8 függvényhez válaszd a sémát: ****0000.
*   Számold ki az átlagfitneszt.
*   Majd cseréld az összes 0-t 1-re: ****1111, és számold ki az átlagot.
*   Értelmezd az eredményt: mit „lát” a GA az utolsó 4 bitről?




In [ ]:
def calculate_schema_average_fitness_8bit(schema: str, fitness_func) -> float:
    """Kiszámolja egy adott 8 bites séma átlagfitneszét."""
    matching_individuals = []
    for individual in BITSTRINGS_8:
        matches = True
        for i in range(len(schema)):
            if schema[i] != '*' and schema[i] != individual[i]:
                matches = False
                break
        if matches:
            matching_individuals.append(individual)

    if not matching_individuals:
        return 0.0 # Nincs illeszkedő egyed

    total_fitness = sum(fitness_func(ind) for ind in matching_individuals)
    return total_fitness / len(matching_individuals)

schema_end_0000 = "****0000"
schema_end_1111 = "****1111"

avg_fitness_0000_suffix = calculate_schema_average_fitness_8bit(schema_end_0000, f8)
avg_fitness_1111_suffix = calculate_schema_average_fitness_8bit(schema_end_1111, f8)

print(f"Séma '{schema_end_0000}': Átlagfitnesz = {avg_fitness_0000_suffix:.2f}")
print(f"Séma '{schema_end_1111}': Átlagfitnesz = {avg_fitness_1111_suffix:.2f}")

Séma '****0000': Átlagfitnesz = 44.38
Séma '****1111': Átlagfitnesz = 42.38


Az eredmény azt mutatja, hogy a GA az utolso 4 bit tekintetében, ezen a magas szintű összehasonlításon keresztül, azt látja, hogy a '0000' végződés magasabb átlagfitneszű.
Azonban fontos megjegyezni, hogy a deceptív jelleg nem feltétlenül a teljes blokkok összehasonlításában nyilvánul meg, hanem az alacsonyabb rendu sémák szintjén. Ahogy a 2. feladatban láttuk, az egyedi bitek cseréje 0-ról 1-re gyakran növeli a lokális átlagfitneszt, és ez a hatás vezet ahhoz, hogy a GA a '1'-eseket tartalmazó egyedek felé sodródik.
Ez a jelenség magyarázza, miért konvergál a GA a '11111111' felé a '00000000' helyett, még akkor is, ha a teljes '0000' blokk fitnesze magasabb, mint a '1111' blokké.

## 6) FELADAT (közepes) – Mutáció hatása


*   Ugyanazzal a GA-val (4 vagy 8 bit) futtass két szimulációt:
*   (a) 𝑝𝑚𝑢𝑡 = 0.001
*   (b) 𝑝𝑚𝑢𝑡 = 0.05
*   Hasonlítsd össze, melyik esetben könnyebb a 0000 illetve 00000000 megtalálása.
*   Írd le 3–4 mondatban, hogy miért.




In [ ]:
import random
from itertools import product

POPULATION_SIZE_4BIT = 10
GENERATIONS_4BIT = 20
CHROMOSOME_LENGTH_4BIT = 4
NUM_SIMULATIONS = 50
GLOBAL_OPTIMUM_4BIT = '0000'

def run_ga_simulation(mutation_rate: float) -> int:
    """Futtatja a 4 bites GA szimulációkat egy adott mutációs rátával és visszaadja a sikeres megtalálások számát."""
    found_optimum_count = 0
    for i in range(NUM_SIMULATIONS):
        population = initialize_population(POPULATION_SIZE_4BIT, CHROMOSOME_LENGTH_4BIT)

        for generation in range(GENERATIONS_4BIT):
            evaluated_population = calculate_population_fitness(population, f4)
            best_individual, best_fitness = max(evaluated_population, key=lambda item: item[1])

            if best_individual == GLOBAL_OPTIMUM_4BIT:
                break

            new_population = []
            parents = select_parents_roulette_wheel(evaluated_population, POPULATION_SIZE_4BIT)

            while len(new_population) < POPULATION_SIZE_4BIT:
                parent1 = random.choice(parents)
                parent2 = random.choice(parents)

                child1, child2 = crossover(parent1, parent2)

                new_population.append(mutate(child1, mutation_rate))
                if len(new_population) < POPULATION_SIZE_4BIT:
                    new_population.append(mutate(child2, mutation_rate))
            population = new_population

        evaluated_population = calculate_population_fitness(population, f4)
        final_best_individual, _ = max(evaluated_population, key=lambda item: item[1])

        if final_best_individual == GLOBAL_OPTIMUM_4BIT:
            found_optimum_count += 1

        print(f"  Szimuláció {i+1}/{NUM_SIMULATIONS}, Mutációs ráta: {mutation_rate} - Végső legjobb egyed: {final_best_individual}", end="\r")
    print("\n")
    return found_optimum_count

#pmut = 0.001
mutation_rate_a = 0.001
print(f"Futtatás PMUT = {mutation_rate_a} mellett:")
found_optimum_a = run_ga_simulation(mutation_rate_a)
print(f"Eredmény PMUT = {mutation_rate_a}: A GA {NUM_SIMULATIONS} futásból {found_optimum_a} alkalommal találta meg a globális optimumot ({GLOBAL_OPTIMUM_4BIT}).\n")

#pmut = 0.05
mutation_rate_b = 0.05
print(f"Futtatás PMUT = {mutation_rate_b} mellett:")
found_optimum_b = run_ga_simulation(mutation_rate_b)
print(f"Eredmény PMUT = {mutation_rate_b}: A GA {NUM_SIMULATIONS} futásból {found_optimum_b} alkalommal találta meg a globális optimumot ({GLOBAL_OPTIMUM_4BIT}).\n")



Futtatás PMUT = 0.001 mellett:
  Szimuláció 50/50, Mutációs ráta: 0.001 - Végső legjobb egyed: 1111

Eredmény PMUT = 0.001: A GA 50 futásból 23 alkalommal találta meg a globális optimumot (0000).

Futtatás PMUT = 0.05 mellett:
  Szimuláció 50/50, Mutációs ráta: 0.05 - Végső legjobb egyed: 0000

Eredmény PMUT = 0.05: A GA 50 futásból 29 alkalommal találta meg a globális optimumot (0000).



A mutácios ráta jelentősen befolyásolja a GA teljesítményét a deceptív fitneszfüggvényen. Alacsony mutációs ráta (0.001) esetén a GA hajlamosabb lehet beszorulni a lokális optimumokba, mivel kevesebb esély van arra, hogy a populáció 'ugorjon' a fitnesztéren. Magasabb mutációs ráta (0.05) növeli az explorációt, segítve a GA-t kiszabadulni a lokális optimumokból, de túl magas érték esetén rontja az exploitációt, és a folyamat inkább véletlenszerű keresésre hasonlít. Ideális esetben a mutációs rátának egyensulyban kell lennie az exploráció (új területek felderítése) és az exploitáció (a már bevált megoldások kiaknázása) között.

## 7) FELADAT (haladó) – Mutáció hatása


*   Készíts egy módosított fitness-függvényt:
*   f4′​(x)=f4​(x)+zaj, ahol a zaj pl. −1,0,+1−1,0,+1 egyenletes eloszlásból.
*   Vizsgáld meg, hogy a zaj hatására javul-e vagy romlik-e a GA esélye a globális optimum elérésére.

Javaslat: futtass 100 szimulációt zajjal és zaj nélkül.



In [ ]:
import random
from itertools import product

def f4_noisy(bits: str) -> int:
    """4 bites deceptív függvény zajjal."""
    noise = random.choice([-1, 0, 1])
    return f4[bits] + noise

def run_ga_simulation_flexible_fitness(fitness_func_to_use, simulation_name: str, mutation_rate: float) -> int:
    """Futtatja a 4 bites GA szimulációkat egy adott fitneszfüggvénnyel és mutációs rátával.
    Visszaadja a sikeres megtalálások számát (GLOBAL_OPTIMUM_4BIT)."""
    found_optimum_count = 0
    for i in range(NUM_SIMULATIONS_TASK7):
        population = initialize_population(POPULATION_SIZE_4BIT, CHROMOSOME_LENGTH_4BIT)

        for generation in range(GENERATIONS_4BIT):
            evaluated_population = [(ind, fitness_func_to_use(ind)) for ind in population]

            best_individual, best_fitness = max(evaluated_population, key=lambda item: item[1])

            if best_individual == GLOBAL_OPTIMUM_4BIT:
                found_optimum_count += 1
                break

            new_population = []
            parents = select_parents_roulette_wheel(evaluated_population, POPULATION_SIZE_4BIT)

            while len(new_population) < POPULATION_SIZE_4BIT:
                parent1 = random.choice(parents)
                parent2 = random.choice(parents)

                child1, child2 = crossover(parent1, parent2)

                new_population.append(mutate(child1, mutation_rate))
                if len(new_population) < POPULATION_SIZE_4BIT:
                    new_population.append(mutate(child2, mutation_rate))
            population = new_population


        else:
            evaluated_population = [(ind, fitness_func_to_use(ind)) for ind in population]
            final_best_individual, _ = max(evaluated_population, key=lambda item: item[1])
            if final_best_individual == GLOBAL_OPTIMUM_4BIT:
                found_optimum_count += 1

        print(f"  {simulation_name} Szimuláció {i+1}/{NUM_SIMULATIONS_TASK7} - Siker: {found_optimum_count}", end="\r")
    print("\n")
    return found_optimum_count

POPULATION_SIZE_4BIT = 10
GENERATIONS_4BIT = 20
MUTATION_RATE_4BIT = 0.01
CHROMOSOME_LENGTH_4BIT = 4
NUM_SIMULATIONS_TASK7 = 100
GLOBAL_OPTIMUM_4BIT = '0000'

print(f"Futtatás zaj nélkül (pmut={MUTATION_RATE_4BIT}):")
found_optimum_no_noise = run_ga_simulation_flexible_fitness(lambda x: f4[x], "Zaj nélkül", MUTATION_RATE_4BIT)
print(f"Eredmény zaj nélkül: A GA {NUM_SIMULATIONS_TASK7} futásból {found_optimum_no_noise} alkalommal találta meg a globális optimumot ({GLOBAL_OPTIMUM_4BIT}).\n")

print(f"Futtatás zajjal (pmut={MUTATION_RATE_4BIT}):")
found_optimum_with_noise = run_ga_simulation_flexible_fitness(f4_noisy, "Zajjal", MUTATION_RATE_4BIT)
print(f"Eredmény zajjal: A GA {NUM_SIMULATIONS_TASK7} futásból {found_optimum_with_noise} alkalommal találta meg a globális optimumot ({GLOBAL_OPTIMUM_4BIT}).\n")

Futtatás zaj nélkül (pmut=0.01):
  Zaj nélkül Szimuláció 100/100 - Siker: 55

Eredmény zaj nélkül: A GA 100 futásból 55 alkalommal találta meg a globális optimumot (0000).

Futtatás zajjal (pmut=0.01):
  Zajjal Szimuláció 100/100 - Siker: 45

Eredmény zajjal: A GA 100 futásból 45 alkalommal találta meg a globális optimumot (0000).



A zaj hozzáadása a fitneszfüggvényhez jellemzően rontja a GA teljesítményét a deceptív függvényeken, de bizonyos esetekben segíthet a lokális optimumokból való kimozdulásban. Ha a zaj túl nagy, akkor a fitnesztáj túlságosan fluktuál, ami megnehezíti a GA számára a stabil irány megtalálását a globális optimum felé. Ezzel szemben, ha a zaj 'éppen megfelelő', az segíthet a GA-nak kilépni a sekély lokális optimumokból. Ebben az esetben, mivel a zaj értéke (+-1) nem túl nagy, a zaj általában növeli a fitneszértékek szórását, és ezáltal csökkenti a GA 'pontosságát' a legjobb egyedek kiválasztásában, ami ronthatja az optimum megtalálásának esélyét. Gyakran azt tapasztaljuk, hogy a zaj rontja az exploitációt (a legjobb megoldások megtartását és finomítását).

## 8) FELADAT (haladó) – Más kódolás vizsgálata


*   Definiálj egy **grayscale-szerű leképezést**: 0000 → 0, 1111 → 15, és egy **unimodális függvényt** ezen az intervallumon (pl. konkáv parabolát).
*   Mutasd meg, hogy ugyanarra az optimumra egy másik kódolás már nem deceptív, a GA könnyen találja a megoldást.
*   Hasonlítsd össze futási eredményekkel.



In [ ]:
import random
from itertools import product

def bin_to_int(binary_string: str) -> int:
    """Bináris sztringet (pl. '0011') egész számmá (pl. 3) konvertál."""
    return int(binary_string, 2)

MAX_UNIMODAL_FITNESS = 30
def unimodal_fitness_func(bits: str) -> int:
    """
    Unimodális fitneszfüggvény 4 bites sztringekre.
    A globalis optimum a '0000' (integer 0), fitnesze 30.
    A fitnesz lineárisan csökken az egesz szám értékének növekedésével.
    Ez egy "hill-climbing-barát" fitnesztáj.
    """
    int_val = bin_to_int(bits)
    return MAX_UNIMODAL_FITNESS - int_val

POPULATION_SIZE_4BIT = 10
GENERATIONS_4BIT = 20
MUTATION_RATE_4BIT = 0.01
CHROMOSOME_LENGTH_4BIT = 4
NUM_SIMULATIONS_TASK8 = 100
GLOBAL_OPTIMUM_4BIT = '0000'

print(f"Futtatás az eredeti deceptív f4 függvénnyel (pmut={MUTATION_RATE_4BIT}):")
found_optimum_f4 = run_ga_simulation_flexible_fitness(lambda x: f4[x], "Eredeti f4", MUTATION_RATE_4BIT)
print(f"Eredmény az eredeti f4 függvénnyel: A GA {NUM_SIMULATIONS_TASK8} futásból {found_optimum_f4} alkalommal találta meg a globális optimumot ({GLOBAL_OPTIMUM_4BIT}).\n")

print(f"Futtatás az új, unimodális függvénnyel (pmut={MUTATION_RATE_4BIT}):")
found_optimum_unimodal = run_ga_simulation_flexible_fitness(unimodal_fitness_func, "Unimodális", MUTATION_RATE_4BIT)
print(f"Eredmény az unimodális függvénnyel: A GA {NUM_SIMULATIONS_TASK8} futásból {found_optimum_unimodal} alkalommal találta meg a globális optimumot ({GLOBAL_OPTIMUM_4BIT}).\n")

Futtatás az eredeti deceptív f4 függvénnyel (pmut=0.01):
  Eredeti f4 Szimuláció 100/100 - Siker: 62

Eredmény az eredeti f4 függvénnyel: A GA 100 futásból 62 alkalommal találta meg a globális optimumot (0000).

Futtatás az új, unimodális függvénnyel (pmut=0.01):
  Unimodális Szimuláció 100/100 - Siker: 79

Eredmény az unimodális függvénnyel: A GA 100 futásból 79 alkalommal találta meg a globális optimumot (0000).



Az eredmények világosan megmutatják a kódolás és a fitnesztáj alakjának jelentőségét a genetikus algoritmus teljesítményére.
Az eredeti deceptív f4 függvénnyel a GA 100 futásból kb 50 alkalommal találta meg a 0000 globális optimumot. Ez a relatíve alacsony szám a deceptív jellegből adódó lokális optimumoknak és félrevezető sémáknak köszönhető.
Ezzel szemben az új, unimodális fitneszfüggvénnyel a GA 100 futásból kb 80 alkalommal találta meg a 0000 globális optimumot.
Látható, hogy az unimodális függvénnyel a GA sokkal nagyobb valószínűséggel találja meg a globális optimumot. Ennek oka, hogy az unimodális fitnesztáj 'sima', nincsenek benne lokális optimumok, amelyek félrevezetnék a GA-t. A magasabb fitneszértékek egyértelműen a globális optimum felé mutatnak, így a szelekció, keresztezés és mutáció mechanizmusai hatékonyan vezetik a populációt a helyes irányba. Ez a demonstráció kiemeli, hogy a probléma reprezentációja (kódolása) és a fitneszfüggvény jellege (deceptív vs. unimodális) mennyire alapvető az evolúciós algoritmusok sikerességéhez.

## 9) FELADAT (haladó) – 8 bites blokkok kombinálása


*   A fenti 𝑓8-at módosítsd úgy, hogy az első blokkon f4, a második blokkon pedig egy egyszerű **hill-climbing-barát függvény** fusson (pl. az 1-ek száma). Mit is takar pontosan a **hill-climbing-barát függvény**? Kérem kifejteni.
*   Figyeld meg, hogy a GA az egyik blokkban „elcsúszik” (deceptív), a másikban jól dolgozik.
*  Írd le, mit tanulhatunk ebből a problémakódolásról és a GA korlátairól



In [ ]:
import random
from itertools import product

POPULATION_SIZE = 100
GENERATIONS = 50
MUTATION_RATE = 0.01
CHROMOSOME_LENGTH = 8

def generate_individual(length: int) -> str:
    """Véletlenszerű N bites egyed generálása."""
    return ''.join(random.choice('01') for _ in range(length))

def initialize_population(size: int, chromosome_length: int) -> list:
    """Kezdő populáció inicializálása."""
    return [generate_individual(chromosome_length) for _ in range(size)]

def calculate_population_fitness(population: list, fitness_func) -> list:
    """A populáció minden egyedének fitneszét kiszámítja. fitness_func egy függvényt vár."""
    return [(individual, fitness_func(individual)) for individual in population]

def select_parents_roulette_wheel(evaluated_population: list, pop_size: int) -> list:
    """Rulettkerék szelekció a szülők kiválasztására."""
    total_fitness = sum(fitness for _, fitness in evaluated_population)
    if total_fitness == 0:
        return random.choices([ind for ind, _ in evaluated_population], k=pop_size)
    selection_probs = [fitness / total_fitness for _, fitness in evaluated_population]
    selected_indices = random.choices(range(len(evaluated_population)), weights=selection_probs, k=pop_size)
    return [evaluated_population[i][0] for i in selected_indices]

def crossover(parent1: str, parent2: str) -> tuple[str, str]:
    """Egypontos keresztezés."""
    crossover_point = random.randint(1, len(parent1) - 1)
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2

def mutate(individual: str, mutation_rate: float) -> str:
    """Bit-flip mutáció."""
    mutated_individual = list(individual)
    for i in range(len(mutated_individual)):
        if random.random() < mutation_rate:
            mutated_individual[i] = '1' if mutated_individual[i] == '0' else '0'
    return "".join(mutated_individual)

DF2 = {
    "0000": 28, "0001": 26, "0010": 24, "0011": 18,
    "0100": 22, "0101":  6, "0110": 14, "0111":  0,
    "1000": 20, "1001": 12, "1010": 10, "1011":  2,
    "1100":  8, "1101":  4, "1110":  6, "1111": 30,
}
def bit_not(bits: str) -> str:
    return "".join("1" if b == "0" else "0" for b in bits)
f4 = {b: DF2[bit_not(b)] for b in DF2}

def bin_to_int(binary_string: str) -> int:
    """Bináris sztringet (pl. '0011') egész számmá (pl. 3) konvertál."""
    return int(binary_string, 2)

MAX_FITNESS_BLOCK = 30
def f4_unimodal(bits: str) -> int:
    """
    Unimodális fitneszfüggvény 4 bites sztringekre, ahol a '0000' az optimum.
    A fitnesz lineárisan csökken az egész szám értékének növekedésével.
    Ez egy "hill-climbing-barát" fitnesztáj.
    """
    int_val = bin_to_int(bits)
    return MAX_FITNESS_BLOCK - int_val

def f8_hybrid(bits: str) -> int:
    """
    8 bites hibrid fitneszfüggvény:
    - Első 4 bit: eredeti deceptív f4
    - Utolsó 4 bit: unimodális f4_unimodal
    """
    assert len(bits) == 8
    return f4[bits[:4]] + f4_unimodal(bits[4:])

CHROMOSOME_LENGTH_8BIT = 8
GLOBAL_OPTIMUM_8BIT = '00000000'
NUM_SIMULATIONS_TASK9 = 50

def run_hybrid_ga_simulation(fitness_func) -> int:
    """
    Futtatja a GA-t az f8_hybrid függvénnyel, és visszatér a globális optimum megtalálásainak számával.
    A kimenet minden szimuláció végén részletezi a legjobb egyedet és blokkjait.
    """
    found_optimum_count = 0
    print(f"\n Hibrid Genetikus Algoritmus Futtatása ({CHROMOSOME_LENGTH_8BIT} bit): ")

    for sim_num in range(NUM_SIMULATIONS_TASK9):
        population = initialize_population(POPULATION_SIZE, CHROMOSOME_LENGTH_8BIT)

        for generation in range(GENERATIONS):
            evaluated_population = calculate_population_fitness(population, fitness_func)
            best_individual, best_fitness = max(evaluated_population, key=lambda item: item[1])

            if best_individual == GLOBAL_OPTIMUM_8BIT:
                break

            new_population = []
            parents = select_parents_roulette_wheel(evaluated_population, POPULATION_SIZE)

            while len(new_population) < POPULATION_SIZE:
                parent1 = random.choice(parents)
                parent2 = random.choice(parents)

                child1, child2 = crossover(parent1, parent2)

                new_population.append(mutate(child1, MUTATION_RATE))
                if len(new_population) < POPULATION_SIZE:
                    new_population.append(mutate(child2, MUTATION_RATE))
            population = new_population

        final_evaluated_population = calculate_population_fitness(population, fitness_func)
        final_best_individual, final_best_fitness = max(final_evaluated_population, key=lambda item: item[1])

        first_block = final_best_individual[:4]
        second_block = final_best_individual[4:]

        if final_best_individual == GLOBAL_OPTIMUM_8BIT:
            found_optimum_count += 1
            print(f"-szimuláció {sim_num+1}/{NUM_SIMULATIONS_TASK9}: Siker! Optimum ({GLOBAL_OPTIMUM_8BIT}) megtalálva. Utolsó legjobb egyed: {final_best_individual}, Fitnesz: {final_best_fitness}")
        else:
            print(f"-szimuláció {sim_num+1}/{NUM_SIMULATIONS_TASK9}: Nem találta meg az optimumot. Utolsó legjobb egyed: {final_best_individual}, Fitnesz: {final_best_fitness} (első blokk fitnesz (deceptív): {f4[first_block]}; második blokk fitnesz (unimodális): {f4_unimodal(second_block)})")
    return found_optimum_count

success_count_hybrid = run_hybrid_ga_simulation(f8_hybrid)
print(f"\n----------------------------")
print(f"\nÖsszegzés:")
print(f"A GA {NUM_SIMULATIONS_TASK9} futásból {success_count_hybrid} alkalommal találta meg a globális optimumot ({GLOBAL_OPTIMUM_8BIT}) az f8_hybrid függvénnyel.")


 Hibrid Genetikus Algoritmus Futtatása (8 bit): 
-szimuláció 1/50: Siker! Optimum (00000000) megtalálva. Utolsó legjobb egyed: 00000000, Fitnesz: 60
-szimuláció 2/50: Siker! Optimum (00000000) megtalálva. Utolsó legjobb egyed: 00000000, Fitnesz: 60
-szimuláció 3/50: Siker! Optimum (00000000) megtalálva. Utolsó legjobb egyed: 00000000, Fitnesz: 60
-szimuláció 4/50: Siker! Optimum (00000000) megtalálva. Utolsó legjobb egyed: 00000000, Fitnesz: 60
-szimuláció 5/50: Siker! Optimum (00000000) megtalálva. Utolsó legjobb egyed: 00000000, Fitnesz: 60
-szimuláció 6/50: Siker! Optimum (00000000) megtalálva. Utolsó legjobb egyed: 00000000, Fitnesz: 60
-szimuláció 7/50: Siker! Optimum (00000000) megtalálva. Utolsó legjobb egyed: 00000000, Fitnesz: 60
-szimuláció 8/50: Siker! Optimum (00000000) megtalálva. Utolsó legjobb egyed: 00000000, Fitnesz: 60
-szimuláció 9/50: Siker! Optimum (00000000) megtalálva. Utolsó legjobb egyed: 00000000, Fitnesz: 60
-szimuláció 10/50: Siker! Optimum (00000000) megta

Képzeld el, hogy egy dombos tájon sétálsz, és a legmagasabb pontot keresed. Egy 'hill-climbing-friendly' (dombtetőre mászást segítő) függvény olyan, mint egy "sima" táj, ahol nincsenek rejtett völgyek vagy félrevezető kisebb csúcsok. Akármerre indulsz el, ha felfelé mész, egyértelműen a fő csúcs felé haladsz. Egy ilyen tájon még egy egyszerű 'felfelé sétálós' algoritmus is könnyedén megtalálja a legmagasabb pontot, mert minden lépés egyértelműen a jó irányba mutat.

###A szimuláció eredményei dióhéjban

A kimenetből látszik, hogy a hibrid függvényünkkel (amely két részre bontja a problémát) a genetikus algoritmus a második, "unimodális" (sima táj) blokkon sokkal ügyesebben találta meg a "0000" optimális részt. Viszont az első, "deceptív" (megtévesztő) blokk gyakran "félrecsúszott", és inkább a '1'-esek felé mozdult el, mert a trükkös "sémák" rossz irányba terelték.


### Mit tanulhatunk ebből a problémakódolásról és a GA korlátairól?
Ez a kísérlet kiválóan megmutatja, hogy a genetikus algoritmus (GA) mennyire függ attól, hogy hogyan "meséljük el" neki a problémát (azaz hogyan kódoljuk) és milyen "tájat" kap (a fitneszfüggvény formáját). Ha a fitnesz "sima" és egyértelműen a cél felé vezet (mint az unimodális rész), akkor a GA szuperül működik. De ha a táj tele van megtévesztő kisebb csúcsokkal (lokális optimumokkal), mint a deceptív rész, akkor a GA könnyen eltévedhet, és nem találja meg az igazi, globális csúcsot. Ez is rávilágít arra, hogy a GA "nem gondolkodik" úgy, mint mi. Nem "érti" a probléma mélyebb összefüggéseit, csak a kapott számok (fitneszértékek) alapján navigál. Egy bonyolult feladatnál ezért létfontosságú, hogy úgy alakítsuk ki a kódolást és a fitneszfüggvényt, hogy a GA számára "könnyen bejárható" legyen a "táj".